In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage.feature import hog
from skimage import io
import warnings
warnings.filterwarnings("ignore")
import glob
import random
import os
from google.colab import drive
!pip install patool
import patoolib

def kmeans_segmentation(image, n_clusters=2):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Reshape the image to a 2D array of pixels
    pixels = image.reshape((-1, 3))
    # Apply K-means clustering
    #Define your own initial centroids
    initial_centroids = np.array([[1, 1, 1], [100, 100, 100]])  # Example initial centroids
    kmeans = KMeans(n_clusters=n_clusters, init=initial_centroids, n_init=1)
    kmeans.fit(pixels)

    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    segmented_img = labels.reshape(image.shape[:2]).astype(np.uint8)
    segmented_img_mask = segmented_img == 1
    return segmented_img_mask

def hog_feature(image):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #print("gray_img: ", gray_img.shape)
    hog_features, hog_image = hog(gray_img, orientations=9, pixels_per_cell=(16, 16),
                cells_per_block=(1, 1), visualize=True, block_norm ='L2-Hys')
    return hog_image

def calculate_iou(seg_mask, gt_mask):
    intersection = np.logical_and(seg_mask, gt_mask)
    union = np.logical_or(seg_mask, gt_mask)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

drive.mount('/content/gdrive/')

patoolib.extract_archive("/content/gdrive/My Drive/VOC2012.zip", outdir="/content/sample_data/")


In [31]:
img_folder_path = "/content/sample_data/VOC2012/JPEGImages"
gt_folder_path = "/content/sample_data/VOC2012/SegmentationClass"

gt_file = [f for f in os.listdir(gt_folder_path)]

basic_iou_list = []
hog_iou_list = []
for i in range(250):
  k = random.randint(0, len(gt_file)-1)
  print("Image: ", i+1)
  print("Random File No: ", k)
  # Load VOC 2012 dataset
  #print("img_file[k]: ",img_file[k])

  file_name = gt_folder_path + "/"+ gt_file[k]
  print("gt_file: ",file_name)
  ground_truth = io.imread(file_name)

  new_file_name = img_folder_path + "/"+ gt_file[k]
  new_file_name = new_file_name.replace("png", "jpg")
  print("img_file: ",new_file_name)

  image = io.imread(new_file_name)

  # Convert the image to grayscale
  gray_image = cv2.cvtColor(ground_truth, cv2.COLOR_RGB2GRAY)
  _, ground_truth_mask = cv2.threshold(gray_image, 10, 255, cv2.THRESH_BINARY)

  # Basic K-means segmentation
  basic_segmented_img = kmeans_segmentation(image)

  #+++++++++++++++++++Improved segmentation using HOG features ++++++++++++++++++++
  hog_image = hog_feature(image)

  #find bounding box including all hog features
  _,hog_mask = cv2.threshold(hog_image, 20, 255, cv2.THRESH_BINARY)

  contours, hierarchy = cv2.findContours(hog_mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

  minX = hog_mask.shape[0]
  minY = hog_mask.shape[1]
  maxX=0
  maxY=0
  # loop over the contours
  for c in contours:
    # get bounding rect
    (x,y,w,h) = cv2.boundingRect(c)
    if(x < minX):
      minX = x
    if(y < minY):
      minY = y
    if(x > maxX):
      maxX = x
    if(y > maxY):
      maxY = y

  hog_mask =cv2.cvtColor(hog_mask.astype(np.uint8),cv2.COLOR_GRAY2RGB)
  cv2.rectangle(hog_mask, (minX,minY), (maxX,maxY), (0, 0, 255), 2)

  #create mask from HoG feature points
  mask = np.zeros_like(hog_mask)
  mask[minY:maxY+1, minX:maxX+1, :] = 255

  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image_rgb = cv2.bitwise_and(image_rgb, mask)
  # Reshape the image to a 2D array of pixels
  pixels_hog = image_rgb.reshape((-1, 3))

  #Define your own initial centroids
  initial_centroids = np.array([[1, 1, 1], [100, 100, 100]])  # Example initial centroids
  kmeans_hog = KMeans(n_clusters=2, init=initial_centroids, n_init=1)
  kmeans_hog.fit(pixels_hog)
  labels_hog = kmeans_hog.labels_
  hog_segmented_img = labels_hog.reshape(image.shape[:2]).astype(np.uint8)

  hog_segmented_img_mask = hog_segmented_img == 1

  # Calculate IoU for basic segmentation
  basic_iou = calculate_iou(basic_segmented_img, ground_truth_mask)

  # Calculate IoU for segmentation using HOG features
  hog_iou = calculate_iou(hog_segmented_img_mask, ground_truth_mask)

  '''
  # Visualization
  plt.figure(figsize=(12, 6))
  plt.subplot(2, 3, 1)
  plt.imshow(image)
  plt.title('Original Image')
  plt.axis('off')

  plt.subplot(2, 3, 2)
  plt.imshow(ground_truth_mask, cmap='gray')
  plt.title('Ground Truth Mask')
  plt.axis('off')


  plt.subplot(2, 3, 3)
  plt.imshow(basic_segmented_img.astype(np.uint8), cmap='gray')
  plt.title('Basic Segmentation')
  plt.axis('off')


  plt.subplot(2, 3, 4)
  plt.imshow(hog_mask,  cmap='gray')
  plt.title('HoG features')
  plt.axis('off')

  plt.subplot(2, 3, 5)
  plt.imshow(hog_segmented_img, cmap='gray')
  plt.title('Improved Segmentation (HOG)')
  plt.axis('off')
  plt.show()
  '''
  print("IoU for Basic Segmentation:", basic_iou)
  print("IoU for Improved Segmentation (HOG):", hog_iou)
  basic_iou_list.append(basic_iou)
  hog_iou_list.append(hog_iou)

# Print IoU scores
print(" All IoU for Basic Segmentation:", basic_iou_list)
print("All IoU for Improved Segmentation (HOG):", hog_iou_list)

print("Average IoU for Basic Segmentation:", np.array(basic_iou_list).mean())
print("Average IoU for Improved Segmentation (HOG):", np.array(hog_iou_list).mean())

Image:  0
Random File No:  1253
gt_file:  /content/sample_data/VOC2012/SegmentationClass/2008_004175.png
img_file:  /content/sample_data/VOC2012/JPEGImages/2008_004175.jpg
IoU for Basic Segmentation: 0.03892579390797148
IoU for Improved Segmentation (HOG): 0.05321660014591648
Image:  1
Random File No:  1483
gt_file:  /content/sample_data/VOC2012/SegmentationClass/2007_007398.png
img_file:  /content/sample_data/VOC2012/JPEGImages/2007_007398.jpg
IoU for Basic Segmentation: 0.2591013010766097
IoU for Improved Segmentation (HOG): 0.23997812788353098
Image:  2
Random File No:  2602
gt_file:  /content/sample_data/VOC2012/SegmentationClass/2007_003349.png
img_file:  /content/sample_data/VOC2012/JPEGImages/2007_003349.jpg
IoU for Basic Segmentation: 0.026410955333774962
IoU for Improved Segmentation (HOG): 0.04120249281539604
Image:  3
Random File No:  101
gt_file:  /content/sample_data/VOC2012/SegmentationClass/2011_001069.png
img_file:  /content/sample_data/VOC2012/JPEGImages/2011_001069.jp